# Ref: Kernels


https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

https://www.kaggle.com/artgor/nn-baseline

https://www.kaggle.com/ogrellier/user-level-lightgbm-lb-1-4480

https://www.kaggle.com/dimitreoliveira/deep-learning-keras-ga-revenue-prediction


# setup and import

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.preprocessing import LabelEncoder

path = '../data/ggl/'
print(os.listdir(path))

['sample_submission.csv', 'train_orig.csv', 'train_parsed.csv', 'test_orig.csv']


# Parse data

In [2]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, 
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

In [5]:
%%time
df_train = load_df(path + 'train_orig.csv')

CPU times: user 1min 47s, sys: 3.06 s, total: 1min 50s
Wall time: 1min 50s


In [6]:
%%time
df_train.to_csv(path + 'train_parsed.csv')

CPU times: user 21 s, sys: 516 ms, total: 21.6 s
Wall time: 22.7 s


In [3]:
%%time
df_test = load_df(path + 'test_orig.csv')

CPU times: user 1min 35s, sys: 2.75 s, total: 1min 38s
Wall time: 1min 37s


In [4]:
%%time
df_test.to_csv(path + 'test_parsed.csv')

CPU times: user 18.9 s, sys: 492 ms, total: 19.4 s
Wall time: 19.7 s


# delete columns with no valid info

In [6]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.preprocessing import LabelEncoder

path = '../data/ggl/'
print(os.listdir(path))

['sample_submission.csv', 'train_orig.csv', 'test_parsed.csv', 'train_parsed.csv', 'test_orig.csv']


In [7]:
%%time
df_train = pd.read_csv(path +'train_parsed.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_test = pd.read_csv(path +'test_parsed.csv', index_col = 0, dtype={'fullVisitorId': 'str'})

/home/kyokicchi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() < 2]
cols_to_drop = [col for col in cols_to_drop if col in df_test.columns]
cols_to_drop

In [11]:
df_train.drop(cols_to_drop, axis=1, inplace=True)
df_test.drop(cols_to_drop, axis=1, inplace=True)

In [12]:
%%time
df_train.to_csv(path + 'train_dropped.csv')
df_test.to_csv(path + 'test_dropped.csv')

CPU times: user 21.9 s, sys: 376 ms, total: 22.2 s
Wall time: 22.6 s


# apply adjustments

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.preprocessing import LabelEncoder

path = '../data/ggl/'
print(os.listdir(path))

['test_dropped.csv', 'sample_submission.csv', 'train_dropped.csv', 'train_orig.csv', 'test_parsed.csv', 'train_parsed.csv', 'test_orig.csv']


In [2]:
%%time
df_train = pd.read_csv(path +'train_dropped.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_test = pd.read_csv(path +'test_dropped.csv', index_col = 0, dtype={'fullVisitorId': 'str'})

<string>:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 8.87 s, sys: 556 ms, total: 9.43 s
Wall time: 9.1 s


## only for train dataset

In [3]:
# to float and log
df_train['transactionRevenue'] = df_train['transactionRevenue'].astype(float)
df_train['transactionRevenue'] = np.log1p(df_train['transactionRevenue'].fillna(0))

df_train.drop('campaignCode', axis=1, inplace=True)


## to both train and test datasets

In [4]:
%%time
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

CPU times: user 3.83 s, sys: 452 ms, total: 4.28 s
Wall time: 3.46 s


### group columns

In [21]:

dont_touch = ['fullVisitorId', 'transactionRevenue',]

bind_cols = ['browser', 'deviceCategory','isMobile','adContent','adwordsClickInfo.adNetworkType','adwordsClickInfo.page',
             'adwordsClickInfo.slot', 'campaign',  'medium']

del_cols = ['sessionId', 'visitId','city','networkDomain','keyword','adwordsClickInfo.gclId','date','dummy','metro','region']

cat_cols = ['country','operatingSystem','subContinent','source']

onehot_cols = ['channelGrouping','continent']

num_cols = ['visitNumber','visitStartTime','hits','pageviews','referralPath']


### analyze date

In [6]:

def dateCol(df):
# fix and breakdown date info
    df['date'] = pd.to_datetime(df['date'].apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:]))
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    df['weekofyear'] = df['date'].dt.weekofyear
    df['weekend'] = df['weekday'].apply(lambda x: 1 if x>=5 else 0)
    
    return df

df_train = dateCol(df_train)
df_test = dateCol(df_test)

### bind and drop

In [7]:
%%time

#bind_cols = ['browser', 'deviceCategory','isMobile','adContent','adwordsClickInfo.adNetworkType','adwordsClickInfo.page',
#             'adwordsClickInfo.slot', 'campaign','campaignCode',  'medium']

df_train['isMobile'] = df_train['isMobile'] * 1
df_train['browser_cat'] = df_train['browser'] + '_' + df_train['deviceCategory'] + '_' + df_train['isMobile'].astype(str)

df_train['ads_1'] = df_train['adContent'].astype(str) + '_' + df_train['adwordsClickInfo.adNetworkType'].astype(str) \
+ '_' + df_train['adwordsClickInfo.page'].astype(str)
df_train['ads_2'] = df_train['adwordsClickInfo.slot'].astype(str) + '_' + df_train['campaign'].astype(str) \
+ '_' + df_train['medium'].astype(str)


df_test['isMobile'] = df_test['isMobile'] * 1
df_test['browser_cat'] = df_test['browser'] + '_' + df_test['deviceCategory'] + '_' + df_test['isMobile'].astype(str)

df_test['ads_1'] = df_test['adContent'].astype(str) + '_' + df_test['adwordsClickInfo.adNetworkType'].astype(str) \
+ '_' + df_test['adwordsClickInfo.page'].astype(str)
df_test['ads_2'] = df_test['adwordsClickInfo.slot'].astype(str) + '_' + df_test['campaign'].astype(str) \
+ '_' + df_test['medium'].astype(str)


for x in bind_cols:
    df_train.drop(x, axis=1, inplace=True)
    df_test.drop(x, axis=1, inplace=True)

CPU times: user 7.17 s, sys: 1.74 s, total: 8.91 s
Wall time: 8.09 s


### access count column

In [8]:
%%time
def userCount(df):
    df['dummy'] = 1
    df['user_cumcnt_per_day'] = (df[['fullVisitorId','date', 'dummy']].groupby(['fullVisitorId','date'])['dummy'].cumcount()+1)
    df['user_sum_per_day'] = df[['fullVisitorId','date', 'dummy']].groupby(['fullVisitorId','date'])['dummy'].transform(sum)
    df['user_cumcnt_sum_ratio_per_day'] = df['user_cumcnt_per_day'] / df['user_sum_per_day'] 

    return df

df_train = userCount(df_train)
df_test = userCount(df_test)

CPU times: user 6.91 s, sys: 528 ms, total: 7.44 s
Wall time: 6.17 s


### category string to numbers

In [10]:
%%time

#cat_cols = ['country','operatingSystem','subContinent','source']
cat_cols.extend(['browser_cat','ads_1','ads_2'])

for col in cat_cols:
    lbl = LabelEncoder()
    lbl.fit(list(df_train[col].values.astype('str')) + list(df_test[col].values.astype('str')))
    df_train[col] = lbl.transform(list(df_train[col].values.astype('str')))
    df_test[col] = lbl.transform(list(df_test[col].values.astype('str')))


CPU times: user 37.6 s, sys: 10.2 s, total: 47.8 s
Wall time: 40 s


### One-Hot columns

In [16]:
%%time
#onehot_cols = ['channelGrouping','continent']

def oneHot(df,col):
    dum = pd.get_dummies(df[col], drop_first = True)
    df = pd.concat((df, dum),axis = 1)
    df = df.drop(col,axis = 1)
    return df

for x in onehot_cols:
    df_train = oneHot(df_train,x)
    df_test = oneHot(df_test,x)


CPU times: user 3.11 s, sys: 2 s, total: 5.11 s
Wall time: 3.03 s


### delete columns

In [23]:
%%time

#del_cols = ['sessionId', 'visitId','city','networkDomain','keyword','adwordsClickInfo.gclId','date','dummy','metro','region']

for x in del_cols:
    df_train.drop(x, axis=1, inplace=True)
    df_test.drop(x, axis=1, inplace=True)



CPU times: user 1.97 s, sys: 1.66 s, total: 3.63 s
Wall time: 1.88 s


In [33]:
%%time

#dont_touch = ['fullVisitorId', 'transactionRevenue',]
l_float = [x for x in df_train.columns if x not in dont_touch]

for x in l_float:
    df_train[x] = df_train[x].values.astype('float')
    df_test[x] = df_test[x].values.astype('float')

ValueError: could not convert string to float: '/yt/about/es-419/'

In [31]:
print(df_train.shape)
for x in df_train.columns.values:
    print(type(df_train[x][0]), x , df_train[x].nunique(), "///",  df_train[x][100])

(903653, 34)
<class 'str'> fullVisitorId 714167 /// 585590266481030647
<class 'numpy.float64'> visitNumber 384 /// 1.0
<class 'numpy.float64'> visitStartTime 887159 /// 1472826458.0
<class 'numpy.float64'> operatingSystem 20 /// 7.0
<class 'numpy.float64'> country 222 /// 9.0
<class 'numpy.float64'> subContinent 23 /// 15.0
<class 'numpy.float64'> hits 274 /// 1.0
<class 'numpy.float64'> pageviews 214 /// 1.0
<class 'numpy.float64'> transactionRevenue 5333 /// 0.0
<class 'int'> referralPath 1476 /// 0
<class 'numpy.int64'> source 380 /// 208
<class 'numpy.int64'> month 12 /// 9
<class 'numpy.int64'> day 31 /// 2
<class 'numpy.int64'> weekday 7 /// 4
<class 'numpy.int64'> weekofyear 52 /// 35
<class 'numpy.int64'> weekend 2 /// 0
<class 'numpy.int64'> browser_cat 106 /// 43
<class 'numpy.int64'> ads_1 64 /// 0
<class 'numpy.int64'> ads_2 20 /// 9
<class 'numpy.int64'> user_cumcnt_per_day 32 /// 1
<class 'numpy.int64'> user_sum_per_day 15 /// 1
<class 'numpy.float64'> user_cumcnt_sum_rat

### save adjusted DFs

In [74]:
%%time
df_train.to_csv(path + 'train_adj.csv')
df_test.to_csv(path + 'test_adj.csv')

# create new dataset: aggregated by users

In [94]:

def aggregate_by_users(df, cat_cols):
    aggs = {
        'transactionRevenue': ['sum', 'size'],
        'hits': ['sum', 'min', 'max', 'mean', 'median'],
        'visitNumber': ['sum', 'min', 'max', 'mean', 'median'],
        'pageviews': ['sum', 'min', 'max', 'mean', 'median'],
        'weekend': ['mean'],
        'isMobile': ['mean'],
    }

    for f in cat_cols + ['day', 'month', 'weekofyear']:
        aggs[f] = ['min', 'max', 'mean', 'median', 'var', 'std']

    users = df.groupby('fullVisitorId').agg(aggs)

    new_columns = [
        k + '_' + agg for k in aggs.keys() for agg in aggs[k]
    ]
    users.columns = new_columns

    return users

In [95]:
%%time
no_use = ['fullVisitorId', 'sessionId', 'visitId', 'visitStartTime']
cat_cols = [x for x in df_train.columns if x not in no_use and type(df_train[x][0]) == str]


df_user_train = aggregate_by_users(df_train, cat_cols)

df_test['transactionRevenue'] = 0
df_user_test = aggregate_by_users(df_test, cat_cols)



CPU times: user 7.03 s, sys: 2.66 s, total: 9.69 s
Wall time: 5.04 s


In [100]:
%%time
df_user_train = df_user_train.fillna(0)
df_user_test = df_user_test.fillna(0)

df_user_train.to_csv(path + 'user_train.csv')
df_user_test.to_csv(path + 'user_test.csv')

CPU times: user 30.7 s, sys: 569 ms, total: 31.3 s
Wall time: 31.2 s


# Normalize

In [7]:
%%time

import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import sklearn.preprocessing as preprocessing

path = '../data/ggl/'

df_user_train = pd.read_csv(path +'user_train.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_user_test = pd.read_csv(path +'user_test.csv', index_col = 0, dtype={'fullVisitorId': 'str'})

print(os.listdir(path))

['user_test.csv', 'train_wip.csv', 'train.csv', 'test_wip.csv', 'user_train.csv', 'test.csv']
CPU times: user 5.48 s, sys: 329 ms, total: 5.81 s
Wall time: 5.51 s


In [8]:
%%time
tgt = ['transactionRevenue_size', 'hits_sum',
       'hits_min', 'hits_max', 'hits_mean', 'hits_median', 'visitNumber_sum',
       'visitNumber_min', 'visitNumber_max', 'visitNumber_mean',
       'visitNumber_median', 'pageviews_sum', 'pageviews_min', 'pageviews_max',
       'pageviews_mean', 'pageviews_median', 'weekend_mean', 'year_mean',
       'isMobile_mean', 'day_min', 'day_max', 'day_mean', 'day_median',
       'day_var', 'day_std', 'month_min', 'month_max', 'month_mean',
       'month_median', 'month_var', 'month_std', 'weekofyear_min',
       'weekofyear_max', 'weekofyear_mean', 'weekofyear_median',
       'weekofyear_var', 'weekofyear_std']

scaler = preprocessing.MinMaxScaler()
df_user_train[tgt] = scaler.fit_transform(df_user_train[tgt])
df_user_test[tgt] = scaler.transform(df_user_test[tgt])

CPU times: user 8.91 s, sys: 1.51 s, total: 10.4 s
Wall time: 5.67 s


In [11]:
%%time
df_user_train.to_csv(path + 'user_train_n.csv')
df_user_test.to_csv(path + 'user_test_n.csv')

CPU times: user 43.9 s, sys: 337 ms, total: 44.2 s
Wall time: 44.2 s
